# Train test split

Take images and labels folder and turn it into train/ and val/ folders.

In [16]:
path_imgs = "/home/jetracer/Documents/3d_mai/data/train_threads_b/overlaying/images"
path_train_val = "/home/jetracer/Documents/3d_mai/data/train_threads_b/overlaying/train_val_split/"

In [17]:
unique_classes = ['dst4', 'm4_10mm', 'din125_m4', 'm4_20mm', 'asy_m4_nut_screw_10mm']

In [18]:
import os
def delete_and_create_folder(folder_path, subdirectories=None):
    try:
        # Check if the folder exists
        if os.path.exists(folder_path):
            # Delete existing folder and its contents
            for root, dirs, files in os.walk(folder_path, topdown=False):
                for file_name in files:
                    file_path = os.path.join(root, file_name)
                    os.remove(file_path)
                for dir_name in dirs:
                    dir_path = os.path.join(root, dir_name)
                    os.rmdir(dir_path)
            os.rmdir(folder_path)
            # print(f"Existing folder '{folder_path}' deleted successfully.")

        # Create a new empty folder
        os.makedirs(folder_path)
        # print(f"New empty folder '{folder_path}' created successfully.")

        # Create subdirectories if specified
        if subdirectories:
            for subdirectory_info in subdirectories:
                subdirectory_name = subdirectory_info["name"]
                subdirectory_path = os.path.join(folder_path, subdirectory_name)
                os.makedirs(subdirectory_path)
                # print(f"Subdirectory '{subdirectory_path}' created.")

                # Recursively create nested subdirectories
                nested_subdirectories = subdirectory_info.get("subdirectories", [])
                if nested_subdirectories:
                    nested_path = subdirectory_path
                    delete_and_create_folder(nested_path, nested_subdirectories)
    except OSError as e:
        print(f"Error: {folder_path} - {e.strerror}")

def copy_file(source_file, destination_directory):
    try:
        # Ensure the destination directory exists
        os.makedirs(destination_directory, exist_ok=True)

        # Extract the file name from the source file path
        file_name = os.path.basename(source_file)

        # Create the destination file path
        destination_file = os.path.join(destination_directory, file_name)

        # Copy the file
        with open(source_file, 'rb') as source, open(destination_file, 'wb') as destination:
            destination.write(source.read())

        print(f"File '{source_file}' copied to '{destination_file}' successfully.")
    except IOError as e:
        print(f"Error: {e}")

In [19]:
import os
def get_files_in_subdirectories(folder_path, file_extension='', file_contains=''):
    files = []
    for root, directories, filenames in os.walk(folder_path):
        for filename in filenames:
            if file_extension == '' and file_contains == '':
                files.append(os.path.join(root, filename))
            elif file_extension != '' and file_contains == '':
                if filename.endswith(file_extension):
                    files.append(os.path.join(root, filename))
            elif file_extension == '' and file_contains != '':
                if file_contains in filename:
                    files.append(os.path.join(root, filename))
            else:
                if file_contains in filename and filename.endswith(file_extension):
                    files.append(os.path.join(root, filename))
    return files

import random
def create_binary_list(n, percentage_of_ones):
    if percentage_of_ones < 0 or percentage_of_ones > 100:
        raise ValueError("Percentage_of_ones must be between 0 and 100")

    num_ones = int(n * (percentage_of_ones / 100))
    num_zeros = n - num_ones

    binary_list = [1] * num_ones + [0] * num_zeros
    random.shuffle(binary_list)

    return binary_list

In [ ]:


images = get_files_in_subdirectories(path_imgs,'.png')
subdirs = ['train','validation']
# Example usage with nested subdirectories
subdirs = [
    {"name": "train", "subdirectories": [{"name": "images"}, {"name": "labels"}]},
    {"name": "validation", "subdirectories": [{"name": "images"}, {"name": "labels"}]}
]
delete_and_create_folder(path_train_val,subdirs)
percentage_train = 80

images_search = [file for file in images] # if class_obj in file]
train_val = create_binary_list(len(images_search),percentage_train)
train = 0
for index_i,image in enumerate(images_search):
    path_label = image.replace('images','labels').replace('png','txt')
    if(train_val[index_i] == 1): #training
        path_save_image = path_train_val+'train/images'
        path_save_label = path_train_val+'train/labels'
    else: # validation
        path_save_image = path_train_val+'validation/images'
        path_save_label = path_train_val+'validation/labels'
    #print(train_val[index_i])
    #print(image)
    #print(path_label)
    #print(path_save_image)
    #print(path_save_label)
    copy_file(image, path_save_image)
    copy_file(path_label, path_save_label)
    train += train_val[index_i]
print(f'train i {train} percentage {100*train/len(images_search)}')
    # print(f'test {train_val[index_i]} class: {class_obj} image: {image}')